# Terminal commands for data access

```
autossh -AtX vschuma@ssh1.mdc-berlin.de ssh vschuma@murphy.mdc-berlin.net
# open the ssh screen and/or type
ssh -NR 6667:localhost:22 vfs@79.197.179.128 (current network ID)

# mount
sshfs -p 6667 vschuma@localhost:/data/local/rajewsky/home/vschuma /mnt/murphy_workspace/
or
sshfs vschuma@murphy:/data/local/rajewsky/home/vschuma/ ~/mounts/murphy_workspace/

# if working with the mount is to slow, copy the data to a ramdisk
sudo mount -t tmpfs -o size=6g tmpfs /mnt/ramdisk/

# then copy the file to the ramdisk
mkdir /mnt/ramdisk/data && rsync -av /mnt/murphy_workspace/NSTT/data/GarciaAlsono_uterus/hot_data/* /mnt/ramdisk/data
or
mkdir /dev/shm/data && rsync -av /home/vschuma/mounts/murphy_workspace/NSTT/data/GarciaAlsono_uterus/hot_data/* /dev/shm/data/
# symlink inside the pycharm project to the data dir
(example command)
ln -s /mnt/ramdisk/data data
e.g.: ln -s /mnt/murphy_workspace/NSTT/data/ data
or
ln -s /dev/shm/data/ data
```

In [1]:
%matplotlib inline
import anndata
import novosparc
import os
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import copy

# Reading data
I got the data from Anna & Tancredi. The spatial file is 6w tumor slide, spatial in-house method processed with spacemake. The other data is annotated single-nuclei data, exported by Anna. Method details - idk.

The data is located in Anna's home directory on the lab server:
`/data/local/rajewsky/home/aantona/novaseq/H5AD_snRNA_ST/`
I created a symlink within my own directory:
`/data/local/rajewsky/home/vschuma/NSTT/data/TNBC`

In [2]:
data_dir = 'data' # symlinked to /data/local/rajewsky/home/vschuma/NSTT/data/TNBC_inhouse

## Single-nuclei data

The `snRNA_TNBC_integreted.h5ad` file is the processed file from Anna which she imported to seurat, did stuff and all the annotations with it and exported back from seurat --> h5ad.

In [30]:
%%time

sn_file = os.path.join(data_dir,"snRNA_integrated.h5ad")
adata_sn = sc.read_h5ad(sn_file) # this is ~10GB RAM
print(adata_sn)

AnnData object with n_obs × n_vars = 26978 × 3000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'percent_MT', 'SCT_snn_res.0.8', 'Seurat_clusters_individual', 'S.Score', 'G2M.Score', 'Phase', 'Bach.id', 'Bach.Avd', 'Bach.BCells', 'Bach.Bsl1', 'Bach.Bsl2', 'Bach.BslG', 'Bach.Cafs', 'Bach.CD41', 'Bach.CD42', 'Bach.CD81', 'Bach.CD82', 'Bach.CD83', 'Bach.cDC1', 'Bach.cDC2', 'Bach.CTLs', 'Bach.CyclingT', 'Bach.Ec1', 'Bach.Ec2', 'Bach.Ec3', 'Bach.Ec4', 'Bach.Fb1', 'Bach.Fb2', 'Bach.Fb3', 'Bach.Fb4', 'Bach.Fb5', 'Bach.Fb6', 'Bach.Fb7', 'Bach.Fb8', 'Bach.Fb9', 'Bach.Hs', 'Bach.Hsp', 'Bach.ILCs', 'Bach.Lp', 'Bach.LpBsl', 'Bach.MastCells', 'Bach.MdC1', 'Bach.MdC2', 'Bach.MdC3', 'Bach.migDC', 'Bach.Mo1', 'Bach.Mo2', 'Bach.Mo3', 'Bach.Neutrophils', 'Bach.NK', 'Bach.pDC', 'Bach.Pericytes1', 'Bach.Pericytes2', 'Bach.PlasmaCells', 'Bach.Swc', 'Bach.Tam1', 'Bach.Tam2', 'Bach.Tm', 'Bach.Tregs', 'Bach.max', 'doublet_score', 'doublet_assignment', 'annotated_idents', '

In [31]:
unneccessary = ['Bach.id', 'Bach.Avd', 'Bach.BCells', 'Bach.Bsl1', 'Bach.Bsl2', 'Bach.BslG', 'Bach.Cafs', 'Bach.CD41', 'Bach.CD42', 'Bach.CD81', 'Bach.CD82', 'Bach.CD83', 'Bach.cDC1', 'Bach.cDC2', 'Bach.CTLs', 'Bach.CyclingT', 'Bach.Ec1', 'Bach.Ec2', 'Bach.Ec3', 'Bach.Ec4', 'Bach.Fb1', 'Bach.Fb2', 'Bach.Fb3', 'Bach.Fb4', 'Bach.Fb5', 'Bach.Fb6', 'Bach.Fb7', 'Bach.Fb8', 'Bach.Fb9', 'Bach.Hs', 'Bach.Hsp', 'Bach.ILCs', 'Bach.Lp', 'Bach.LpBsl', 'Bach.MastCells', 'Bach.MdC1', 'Bach.MdC2', 'Bach.MdC3', 'Bach.migDC', 'Bach.Mo1', 'Bach.Mo2', 'Bach.Mo3', 'Bach.Neutrophils', 'Bach.NK', 'Bach.pDC', 'Bach.Pericytes1', 'Bach.Pericytes2', 'Bach.PlasmaCells', 'Bach.Swc', 'Bach.Tam1', 'Bach.Tam2', 'Bach.Tm', 'Bach.Tregs', 'Bach.max','Bach.luminal.epithelial.cell.of.mammary.gland', 'Bach.B.cell', 'Bach.stromal.cell', 'Bach.T.cell', 'Bach.endothelial.cell', 'Bach.basal.cell', 'Bach.macrophage', 'integrated_snn_res.0.8', 'Seurat_clusters_integrated.08','integrated_snn_res.2.5', 'integrated_snn_res.3', 'integrated_snn_res.4','integrated_snn_res.0.9', 'Seurat_clusters_integrated.1', 'integrated_snn_res.1', 'integrated_snn_res.2', 'Seurat_clusters_integrated.2', 'integrated_snn_res.0.4', 'Seurat_clusters_integrated.04', 'integrated_snn_res.0.2','nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'percent_MT', 'SCT_snn_res.0.8', 'Seurat_clusters_individual', 'S.Score', 'G2M.Score', 'doublet_score', 'doublet_assignment', 'condition', 'annotated_Cellcompartments', 'annotated_Celltypes', 'annotated_Cellcompartments2', 'seurat_clusters', 'annotated_CelltypesState.res.2', 'annotated_compartment.res.2', 'annotated_compartmentCellType.res.2', 'celltype_res2.condition', 'celltype_res2', 'annotated_compartmentCellType2.res.2','ConditionGlobal', 'copykat.pred','copyKAT_epithelium_winSize5', 'copyKAT_epithelium_winSize25', 'copycat_pred_epithelium_spearman_winSize_25', 'copycat_7week']

In [32]:
for element in unneccessary:
    if element in adata_sn.obs.columns:
        del adata_sn.obs[element]

In [33]:
adata_sn

AnnData object with n_obs × n_vars = 26978 × 3000
    obs: 'orig.ident', 'Phase', 'annotated_idents', 'Reannotation_detailed', 'Reannotation_global', 'celltype.condition', 'Reannotation_compartemnt', 'ReannotationGlobal', 'ReannotationDetailed'
    var: 'features', 'integrated_features'
    obsm: 'X_umap'
    layers: 'integrated'

In [15]:
adata_sn.obs["ReannotationGlobal"]

AAACCCAAGCTTCATG-1_1    2
AAACCCAAGCTTGTGT-1_1    0
AAACCCAAGGCTTTCA-1_1    0
AAACCCACAACAACAA-1_1    0
AAACCCACAATACGCT-1_1    0
                       ..
TTTGTTGAGCAGGCTA-1_3    0
TTTGTTGAGGTCGACA-1_3    2
TTTGTTGAGTGTAGTA-1_3    0
TTTGTTGCAGAACTAA-1_3    0
TTTGTTGGTCTCGACG-1_3    1
Name: ReannotationGlobal, Length: 26978, dtype: int32

# Create Subsamples of various size

In [26]:
# outfile = 'output/TNBC_sn_reduced.h5ad'
# # there seems to be some naming issue coming from seurat, found this hack at: https://github.com/theislab/scvelo/issues/255
# adata_reduced.__dict__['_raw'].__dict__['_var'] = adata_reduced.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
# adata_reduced.write(outfile)